In [ ]:
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

from utils import *

import os
import json
from datetime import datetime

# 기본 설정
names = ['이재명', '김문수', '이준석']
press_code = {
    '경향신문': '1032',
    '동아일보': '1020',
    '오마이뉴스': '1047',
    '조선일보': '1023',
    '중앙일보': '1025',
    '한겨례신문': '1028'
}

# 날짜 설정!!
start_date = datetime(2025, 4, 27)
end_date = datetime(2025, 6, 3)

In [2]:
def get_meta_data(names, press_code, start_date, end_date):
    """
    셀레니움으로 스크롤 끝까지 내려서 해당하는 날짜의 모든 기사 div 로딩 후, 
    각 기사의 url을 저장하여 json으로 메타데이터 저장
    """
    start_date_str = start_date.strftime('%Y.%m.%d')
    end_date_str = end_date.strftime('%Y.%m.%d')

    # 크롬드라이버 설정 (셀레니움)
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # 필요시 주석 해제
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=chrome_options)

    # 결과 저장용 딕셔너리
    result = {}

    # 후보별, 날짜별, 언론사별 크롤링
    for name in names:
        result[name] = {}
        curr_date = start_date
        while curr_date <= end_date:
            ds = curr_date.strftime('%Y.%m.%d')
            target_date = curr_date.strftime('%Y%m%d')
            for press_name, code in press_code.items():
                if press_name not in result[name]:
                    result[name][press_name] = {'네이버링크': [], '언론사링크': []}

                url = (
                    f"https://search.naver.com/search.naver?"
                    f"ssc=tab.news.all&query={name}&sm=tab_opt&sort=0&photo=0&field=0"
                    f"&pd=3&ds={ds}&de={ds}"
                    f"&docid=&related=0&mynews=1"
                    f"&office_type=1&office_section_code=4"
                    f"&news_office_checked={code}"
                    f"&nso=so%3Ar%2Cp%3Afrom{target_date}to{target_date}"
                    f"&is_sug_officeid=0&office_category=0&service_area="
                )

                print(f"\n=== [{name} - {press_name} - {ds}] ===")
                driver.get(url)
                time.sleep(2)

                # 무한 스크롤
                prev_len = 0
                while True:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(1.5)
                    elements = driver.find_elements(By.CLASS_NAME, "group_news")
                    curr_len = len(elements)
                    if curr_len == prev_len:
                        break
                    prev_len = curr_len

                # 링크 수집
                links = driver.find_elements(By.XPATH, '//div[contains(@class, "group_news")]//a[@href and @target="_blank"]')
                for link in links:
                    href = link.get_attribute("href")
                    if not href or not href.startswith("http"):
                        continue
                    if "n.news.naver.com" in href:
                        if href not in result[name][press_name]['네이버링크']:
                            result[name][press_name]['네이버링크'].append(href)
                    else:
                        if href not in result[name][press_name]['언론사링크']:
                            result[name][press_name]['언론사링크'].append(href)

            curr_date += timedelta(days=1)

    driver.quit()
    save_json(result, f'meta_data_{start_date_str}_{end_date_str}.json')

def fetch_and_parse(start_date, end_date):
    """
    수집한 메타데이터에 대해 url fetch하여 html 수신 후 파싱하여 저장
    """
    start_date_str = start_date.strftime('%Y.%m.%d')
    end_date_str = end_date.strftime('%Y.%m.%d')
    meta_data = load_json(f'meta_data_{start_date_str}_{end_date_str}.json')
    data_store_dict = {}
    for name in names:
        by_candidate_dict = meta_data[name]
        press_list = list(by_candidate_dict.keys())
        for press in press_list:
            try:
                target_url_list = by_candidate_dict[press]['네이버링크']
            except:
                target_url_list = by_candidate_dict[press]['언론사링크']

            for i, url in enumerate(target_url_list):
                print(f"=== {name} | {press} | ({i+1}/{len(target_url_list)}) ===")
                article_data = parse_crawled(url)

                date = article_data['date'][:10].replace('.', '')
                print(date)
                
                custom_by_RAL = {
                    'id': f"{name}_{press}_{date}_{i}",
                    'order': i,
                    "title": article_data['title'],
                    "press": press,
                    "date": article_data['date'],
                    "content": article_data['content'],
                    "source": article_data['source']
                }

                if name not in list(data_store_dict.keys()):
                    data_store_dict[name] = [custom_by_RAL]
                else:
                    data_store_dict[name].append(custom_by_RAL)

    save_json(data_store_dict, f'news_{start_date_str}_{end_date_str}.json')
    print('='*50)
    print('json 저장 완료!')
    print('='*50)

In [3]:
def main(names, press_code, start_date, end_date):
    print("-"*50)
    print(f"{start_date.strftime('%Y.%m.%d')}부터 {end_date.strftime('%Y.%m.%d')}까지")
    print(f'{names} 후보에 대해')
    print(f'{list(press_code.keys())} 언론사에서')
    print('뉴스를 수집합니다.')
    print('\n')
    print('-'*50)
    print('[DEBUG] | 메타데이터 수집 시작...')
    get_meta_data(names, press_code, start_date, end_date)
    print('[DEBUG] | 메타데이터 수집 완료!')
    print('\n')
    print('-'*50)
    print('[DEBUG] | 뉴스 데이터 수집 시작...')
    fetch_and_parse(start_date, end_date)
    print('[DEBUG] | 뉴스 데이터 수집 완료!')

In [4]:
if __name__ == '__main__':
    main(names, press_code, start_date, end_date)

--------------------------------------------------
2025.04.27부터 2025.06.03까지
['이재명', '김문수', '이준석'] 후보에 대해
['경향신문', '동아일보', '오마이뉴스', '조선일보', '중앙일보', '한겨례신문'] 언론사에서
뉴스를 수집합니다.


--------------------------------------------------
[DEBUG] | 메타데이터 수집 시작...

=== [이재명 - 경향신문 - 2025.04.27] ===

=== [이재명 - 동아일보 - 2025.04.27] ===

=== [이재명 - 오마이뉴스 - 2025.04.27] ===

=== [이재명 - 조선일보 - 2025.04.27] ===

=== [이재명 - 중앙일보 - 2025.04.27] ===

=== [이재명 - 한겨례신문 - 2025.04.27] ===

=== [이재명 - 경향신문 - 2025.04.28] ===

=== [이재명 - 동아일보 - 2025.04.28] ===

=== [이재명 - 오마이뉴스 - 2025.04.28] ===

=== [이재명 - 조선일보 - 2025.04.28] ===

=== [이재명 - 중앙일보 - 2025.04.28] ===

=== [이재명 - 한겨례신문 - 2025.04.28] ===

=== [이재명 - 경향신문 - 2025.04.29] ===

=== [이재명 - 동아일보 - 2025.04.29] ===

=== [이재명 - 오마이뉴스 - 2025.04.29] ===

=== [이재명 - 조선일보 - 2025.04.29] ===

=== [이재명 - 중앙일보 - 2025.04.29] ===

=== [이재명 - 한겨례신문 - 2025.04.29] ===

=== [이재명 - 경향신문 - 2025.04.30] ===

=== [이재명 - 동아일보 - 2025.04.30] ===

=== [이재명 - 오마이뉴스 - 2025.04.30] ===

=== [

In [ ]:
def preprocess_and_save(data_dict, output_dir, batch_size=1000):
    """
    후보별로 JSON 파일을 생성하고, 각 파일에 2025년 5월 28일부터 과거 순으로
    하루씩 내려가며 최대 batch_size(기본 1000)개 기사만 저장
    """
    os.makedirs(output_dir, exist_ok=True)

    # 기준 날짜 (최신 시작점)
    pivot_date = datetime(2025, 5, 29)

    for candidate, articles in data_dict.items():
        # 날짜 파싱 함수
        def parse_date(d):
            s = d.replace('오전', 'AM').replace('오후', 'PM')
            return datetime.strptime(s, '%Y.%m.%d. %p %I:%M')

        # 날짜 기준 정렬 (최신 → 과거)
        sorted_articles = sorted(
            articles,
            key=lambda x: parse_date(x['date']),
            reverse=True
        )

        # 5월 28일 이후는 무시하고, 과거로 모으며 1000개까지만
        selected = []
        for article in sorted_articles:
            article_date = parse_date(article['date'])
            if article_date > pivot_date:
                continue
            selected.append(article)
            if len(selected) >= batch_size:
                break

        # 파일 저장
        file_path = os.path.join(output_dir, f"{candidate}.json")
        with open(file_path, 'w', encoding='utf-8') as fp:
            json.dump(selected, fp, ensure_ascii=False, indent=2)

        print(f"Saved {len(selected)} articles for {candidate} -> {file_path}")

In [ ]:
data_dict = load_json('news_2025.04.27_2025.06.03.json')
output_dir = './per_candidate_json'

# 각 후보별로 최신부터 과거 순으로 최대 1000개만 저장
preprocess_and_save(data_dict, output_dir, batch_size=1000)

Saved 1000 articles for 이재명 -> ./per_candidate_json/이재명.json
Saved 1000 articles for 김문수 -> ./per_candidate_json/김문수.json
Saved 1000 articles for 이준석 -> ./per_candidate_json/이준석.json
